## COPD Exacerbation - Prescription of COPD-specific antibiotic combined with OCS for 5–14 days

One aspect of a strategy to identify COPD exacerbations is to use prescription of COPD-specific antibiotic combined with oral corticosteroid for 5–14 days. This strategy is described in [_Validation of the Recording of Acute
Exacerbations of COPD in UK Primary Care Electronic Healthcare Record_](https://journals.plos.org/plosone/article/file?id=10.1371/journal.pone.0151357&type=printable) by Rothie et al (2016). We have seperate codelists for each componen

The following notebooks contains SnoMed/[NHS Dictionary of Medicines and Devices](https://ebmdatalab.net/what-is-the-dmd-the-nhs-dictionary-of-medicines-and-devices/) codes to identify these medicines in a electronic health record. When applying to an EHR we should identify presence of a steroid and antibiotic issued on the same day. NICE currently recommends five days but we will use a week to be pragmatic in case a GP has chose to give slightly longer. We should also look for 10-14 days as some practices may issue two rescue packs on a single prescription which is aproach used in the study above.

- [Steroids](#pred)
- [Antibiotics](#abx)

#### A note on COPD "Rescue Packs"
A COPD "rescue pack" is a prescription for a short course of steroids and antibiotics that are issued in advance and a person with COPD keeps on-hand to use when they identify and self-diagnose an exacerbation in line with a their management plan. A [Cochrane review](https://www.cochrane.org/CD011682/AIRWAYS_self-management-interventions-including-action-plans-patients-chronic-obstructive-pulmonary-disease) has found  _the number of people who had at least one hospital admission related to lung disease was reduced among those who participated in a self-management intervention (moderate-quality evidence)_. In the event of an exacerbation [NICE recommends prescription](https://www.nice.org.uk/guidance/ng115/resources/chronic-obstructive-pulmonary-disease-in-over-16s-diagnosis-and-management-pdf-66141600098245) of a short course of oral steroids and antibiotics to treat an exacerbation. Where appropriate these may be issued in advance to people with self management plans. As rescue packs are given in advance we cannot take the date of their issue as definitely the date of exacerbation. They would most likely be given out at COPD annual review or have asscoiated code like `Anticipatory medicine supply procedures simple reference set` in the record.


In [1]:
#import libraries
from ebmdatalab import bq
import os
import pandas as pd

## Steroid <a id='pred'></a>

NICE guidance advises 30 mg oral prednisolone once daily for 5 days.

We have [defined oral steroids in a here](https://github.com/ebmdatalab/steroids-covid-codelist-notebook/blob/master/notebooks/oral.pred.ipynb) for a seperate study.



## Antibioitcs <a id='abx'></a>

Amoxicillin, Doxycycline and Clarithromycin are generally the first choice antibiotics [recommended by NICE](https://cks.nice.org.uk/chronic-obstructive-pulmonary-disease#!scenarioRecommendation:16) and other local organisations for COPD exacerbations. NICE does recommend using sputum cultures to inform choice of antibiotic so for individual cases other antibiotics may be used. Rothie et al (linked above) do have a list which we will include below. 

In [2]:
sql = '''
WITH bnf_codes AS (
  SELECT bnf_code FROM hscic.presentation WHERE 
  (bnf_code LIKE '0501013%'   OR #bnf antibacterial penicillins
   bnf_code LIKE '050102%'    OR #bnf antibacterial cephalosporins
   bnf_code LIKE '0501050%'   OR #bnf antibacterial macrolides
   bnf_code LIKE '0501120L0%' OR #bnf antibacterial ciprofl
   bnf_code LIKE '0501120X0%' OR #bnf antibacterial mlevofloxacin
   bnf_code LIKE '0501030%')    #bnf antibacterials tetracyclines)
   AND
   (bnf_code NOT LIKE "0501030Z0%AA" AND #doxycyline rosacea prep
    bnf_code NOT LIKE "0501021M0%"   AND #cefradrine
    bnf_code NOT LIKE "0501050A0%"   AND #Azithromycin
    bnf_code NOT LIKE "0501013E0%"   AND #Ampicillin
    bnf_code NOT LIKE "0501013L0%"   AND #co-fluamp
    bnf_code NOT LIKE "0501021C0%"   AND #Azithromycin
    bnf_code NOT LIKE "0501050A0%"   AND #Azithromycin
   
   
   
    Cefixime
    cefradroxil
    bnf_code NOT LIKE "0501050T0%")      #telithromycin

)

SELECT *
FROM measures.dmd_objs_with_form_route
WHERE bnf_code IN (SELECT * FROM bnf_codes) 
AND 
obj_type IN ('vmp', 'amp')
AND
form_route LIKE '%.oral%' 
ORDER BY obj_type, bnf_name, bnf_code, snomed_id
'''


copd_abx_codelist = bq.cached_read(sql, csv_path=os.path.join('..','data','copd_abx_codelist.csv'))
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
copd_abx_codelist

,obj_type,vpid,snomed_id,bnf_code,dmd_name,bnf_name,form_route
0,amp,36565111000001106,18248711000001106,0501030P0BHABAC,Acnamino MR 100mg capsules (Almus Pharmaceuticals Ltd),Acnamino MR 100mg capsules (Almus),capsulemodified-release.oral
1,amp,36565111000001106,9361611000001109,0501030P0BHAAAC,Acnamino MR 100mg capsules (Dexcel-Pharma Ltd),Acnamino MR 100mg capsules (Dexcel-Pharma),capsulemodified-release.oral
2,amp,324091007,147511000001107,0501030P0BDABAE,Aknemin 100mg capsules (Almirall Ltd),Aknemin 100mg capsules,capsule.oral
3,amp,324090008,116911000001108,0501030P0BDAAAD,Aknemin 50 capsules (Almirall Ltd),Aknemin 50 capsules,capsule.oral
4,amp,35901911000001104,42011000001106,0501013B0BGACAS,Amoram 125mg/5ml oral suspension (LPC Medical (UK) Ltd),Amoram 125mg/5ml oral suspension,suspension.oral
5,amp,323509004,500511000001107,0501013B0BGAAAA,Amoram 250mg capsules (LPC Medical (UK) Ltd),Amoram 250mg capsules,capsule.oral
6,amp,323740008,284811000001105,0501013B0BGADAT,Amoram 250mg/5ml oral suspension (LPC Medical (UK) Ltd),Amoram 250mg/5ml oral suspension,suspension.oral
7,amp,323510009,19211000001104,0501013B0BGABAB,Amoram 500mg capsules (LPC Medical (UK) Ltd),Amoram 500mg capsules,capsule.oral
8,amp,323732002,143211000001100,0501013B0AAAJAJ,Amoxicillin 125mg/5ml oral suspension (IVAX Pharmaceuticals UK Ltd),Amoxicillin 125mg/5ml oral suspension,suspension.oral
9,amp,323732002,144711000001109,0501013B0AAAJAJ,Amoxicillin 125mg/5ml oral suspension (Ranbaxy (UK) Ltd),Amoxicillin 125mg/5ml oral suspension,suspension.oral
